In [ ]:
# THIS IS THE FILE THAT WE WILL USE TO CREATE THE CODE THAT WILL BE UPDATED ON TO THE CPP APP

In [52]:
from openpyxl import Workbook
from datetime import datetime 
from openpyxl.styles import PatternFill, Font, Alignment, NamedStyle, Border, Side
import os
import pandas as pd

COLOR_PALETTE = {
    "maroon": "C00000",

    #Banner Colors
    "lavender":"D5B8EA",
    "pink": "FF8FFF",
    "blue": "BDD7F1",
    "peach": "F9D2BD",
    "green": "D6EDBD",
    "cyan": "D9FFFF",
    
    }

def apply_color(cell, color_name):
    """Applies a predefined color to a cell."""
    color_code = COLOR_PALETTE.get(color_name, "FFFFFF")  # Default to white if not found
    fill = PatternFill(start_color=color_code, end_color=color_code, fill_type="solid")
    cell.fill = fill


def get_banner_color(curr_year):
    """This functions inputs the current year and returns the color required for the banner

    Args:
        curr_year (String): current year

    Returns:
        String: respective color for year
    """
    end=int(curr_year[-1])
    if end == 4 or end == 9:
        return "green"

    elif end == 5 or end == 0:
        return "pink"

    elif end == 6  or end == 1:
        return "blue"

    elif end == 7  or end == 2 :
        return "peach"

    elif end == 8 or end == 3 :
        return "lavender"

def create_salaries_tab(wb, params_df):

    #GLOBAL VARIABLES 
    year2_string = params_df.loc[params_df["Parameters"] == "year 2 string", "Values"].iloc[0]
    ws = wb.active
    ws.title = f"{year2_string} SALARIES"
    ws.sheet_properties.tabColor = "C00000"

    # FORMATING STYLES

    percentage_style = NamedStyle(name="percentage")
    percentage_style.number_format = '0.00%'

    date_style = NamedStyle(name="date_style")
    date_style.number_format = 'yyyy-mm-dd;@'

    # accounting formating
    accounting_style = '_("$"* #,##0.00_);_("$"* \\(#,##0.00\\);_("$"* "-"??_);_(@_)'

    general_style = NamedStyle(name="general")
    general_style.number_format = 'General'

    # COLUMN DIMENSION PRESETS

    column_dimensions = [
        ("A", 13.86), ("B", 21.86), ("C", 11.43), ("D", 11.43), ("E", 16.14), 
        ("F", 19), ("G", 16.14), ("H", 9.86), ("I", 14.14), ("J", 22.86), 
        ("K", 14.29), ("L", 14.29), ("M", 14.29), ("N", 2), ("O", 14.29), ("P", 2),
        ("Q", 19.20),("R", 5.57),("S", 53.29)
    ]

    for col_name, col_width in column_dimensions:
        ws.column_dimensions[col_name].width = col_width

    # TITLE SETTINGS
    ws.row_dimensions[3].height= 28.50
    title_cell = ws.cell(row=3, column=1)
    title_cell.value = f"{year2_string} SALARY & FRINGE W/COLAS"
    title_cell.font = Font(size=22, bold=True)

    # TIME STAMP SETTINGS
    time_stamp_cell = ws.cell(row=3, column=8)
    apply_color(time_stamp_cell, "maroon")
    time_stamp_cell.value = "Generated on " + datetime.now().strftime("%b-%d-%Y") + " at " + datetime.now().strftime("%H:%M:%S")
    time_stamp_cell.font = Font(size=14, color="FFFFFF", bold=True)  
    ws.merge_cells(start_row=3, start_column=8, end_row=3, end_column=10)

    # FY BANNER SETTINGS
    ws.row_dimensions[1].height = 50.25
    banner =""
    for i in range(1,30):
        if i % 2 == 0:
            banner = banner + "    -    "
        else:
            banner = banner + year2_string

    banner_cell = ws.cell(row=1, column=1)
    banner_cell.value = banner
    banner_cell.font=Font(name = "Ariel", size=18, color= "2F75B5" , bold=  True)
    banner_cell.alignment = Alignment(vertical="center")

    # banner Color
    for col in range(1, 25):
        cell = ws.cell(row=1, column=col)
        apply_color(cell,get_banner_color(year2_string))

    # PRESETS FOR LINES 5-9

    font_blue59 = Font(color="0000CC",bold=  True)

    cell_presets = {
    5: {
        1: {"value": f"{year2_string} Fringe - Academic Staff", "font": font_blue59},                                                               # Column A (1)
        3: {"value": params_df.loc[params_df["Parameters"] == "academic_fringe", "Values"].iloc[0], "style": percentage_style, "fill": "cyan"},     # Column C (3)
        6: {"value": f"{year2_string} Fiscal Year Start", "font": font_blue59},                                                                     # Column F (6)
        7: {"value": params_df.loc[params_df["Parameters"] == "year_start", "Values"].iloc[0], "style": date_style, "fill": "cyan"},                # Column G (7)
    },
    6: {
        1: {"value": f"{year2_string} Fringe - University Staff", "font": font_blue59},                                                             # Column A (1)
        3: {"value": params_df.loc[params_df["Parameters"] == "university_fringe", "Values"].iloc[0], "style": percentage_style, "fill": "cyan"},   # Column C (3)
        6: {"value": f"{year2_string} Fiscal Year End", "font": font_blue59},                                                                       # Column F (6)
        7: {"value": params_df.loc[params_df["Parameters"] == "year_end", "Values"].iloc[0], "style": date_style, "fill": "cyan"},                  # Column G (7)
    },
    7: {
        1: {"value": f"{year2_string} Fringe - Student Staff", "font": font_blue59},                                                                # Column A (1)
        3: {"value": params_df.loc[params_df["Parameters"] == "student_fringe", "Values"].iloc[0], "style": percentage_style, "fill": "cyan"},      # Column C (3)
        6: {"value": f"{year2_string} # Total Days", "font": font_blue59},                                                                          # Column F (6)
        7: {"value": "=(G6-G5)+1", "fill": "cyan"},                                                                                                 # Column G (7)
    },
    9: {
        6: {"value": f"{year2_string} PAY PERIODS", "font": font_blue59},                                                                           # Column F (6)
        7: {"value": params_df.loc[params_df["Parameters"] == "number of pay periods", "Values"].iloc[0], "fill": "cyan"},                          # Column G (7)
    },
    }

    # Iterate over the dictionary to apply settings
    for row, columns in cell_presets.items():
        for col, settings in columns.items():
            cell = ws.cell(row=row, column=col)  # Use numeric column directly
            if "value" in settings:
                cell.value = settings["value"]
            if "font" in settings:
                cell.font = settings["font"]
            if "style" in settings:
                cell.style = settings["style"]
            if "fill" in settings:
                apply_color(cell, settings["fill"])

    
    
    return

In [53]:
# This is the code that will go in the MAIN CODE GENERATION SECTION

from openpyxl import Workbook
from datetime import datetime 
import os
import pandas as pd
# HARD CODED VALUES THAT NEED TO BE MADE AVAILABLE

# NOTE: for reference always use the second year to name the all sheet FY's. example, if the 2 years are 2024 and 2025 it would be FY25

def create_payroll_spreadsheet(destination_path, params_df):
    """ This is the entry point for the Payroll Spreadsheet all subfunctions are called from here

    Args:
        destination_path (String): Path of the save destination for the payroll spreadsheet
        params_df (Pandas df): Contains all the paramneters and values for the spreadsheet

    Returns:
        String: Saved location of the Payroll Sheet 
    """
    wb = Workbook()
    wb.active
    create_salaries_tab(wb, params_df)
    create_staff_tab(wb, params_df)
    final_save_path = os.path.join(destination_path,get_save_name(f"{params_df.loc[params_df["Parameters"] == "year 2 string", "Values"].iloc[0]} Payroll_Spreadsheet"))
    wb.save(final_save_path)
    wb.close()
    return final_save_path


def create_staff_tab(wb, params_df):
    ws= wb.create_sheet(title =f"{params_df.loc[params_df["Parameters"] == "year 2 string", "Values"].iloc[0]} STAFF")
    ws.sheet_properties.tabColor = "0000CC"
    return

def get_save_name(spreadsheet_name):
    """ This Function forms the new file name using the nomencalture which is {filename_currentDate_cuurentTime)

    Args:
        spreadsheet_name (String): Spreadsheet Base name

    Returns:
        String: Formatted name of file path to be saved
    """
    current_datetime = datetime.now().strftime("%b-%d-%Y_%H-%M-%S")
    return f"{spreadsheet_name}_{current_datetime}.xlsx"


In [54]:
# THIS IS THE CODE THAT CALLS THE GENERATINO FUNCTION
import pandas as pd
num_pay_periods = 25
y1 = 2024
y2 = 2025

data = {
    "Parameters": ["number of pay periods", "year 1 numeric" , "year 1 string", "year 2 numeric" , "year 2 string","academic_fringe",
                   "university_fringe","student_fringe","year_start","year_end"],
    "Values": [f"{num_pay_periods}" , f"{y1 % 100}", f"FY {y1 % 100}",  f"{y2 % 100}", f"FY {y2 % 100}","N/A Too be filled",
               "N/A Too be filled","N/A Too be filled","N/A Too be filled","N/A Too be filled"]
}
df = pd.DataFrame(data)
temporary_save_path = r"C:\Users\dpaul5\Desktop\2025 APP TESTING"  # Corrected path
os.startfile(create_payroll_spreadsheet(temporary_save_path, df))


df



,Parameters,Values
0,number of pay periods,25
1,year 1 numeric,24
2,year 1 string,FY 24
3,year 2 numeric,25
4,year 2 string,FY 25
5,academic_fringe,N/A Too be filled
6,university_fringe,N/A Too be filled
7,student_fringe,N/A Too be filled
8,year_start,N/A Too be filled
9,year_end,N/A Too be filled
